# 01_Split_Videos.ipynb

Original code from Carolyn Parkinson & modified for personal use

Creates fslroi command to split whole run into individual videos

Input: 

* Run Ref File
    
    *  /Users/mdclark/Desktop/EmpAcc/data/behavioraldata/scanfiles/239_run_ref.csv
    * Contains "sub_id, trial, video_number, self_other, neg_pos, scan_start, video_start, video_start_rts, run"
   
* Run File
    
Output: 
   
* Video run file

In [1]:
import os
import subprocess
import pandas as pd

from nilearn import input_data
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# define a function to execute shell commands
def sh(c):
    '''
    run shell commands
    '''
    subprocess.call(c, shell = True)
    
import math

def round_up_to_even(f):
    return math.ceil(f / 2.) * 2

In [ ]:
# get subject data

subjects = ['145', '157', '168', '184', '186', '200', '214', '216', '219', '220', '222', '223', '229', '245', '249', '253', '257', '263', '265', '267', '270', '277', '280', '282', '288']
for sub in subjects: 
    subject = "sub-" + sub
    for run in range(1,5): 

        # set filepaths
        data_filepath = os.path.join("./data/", subject, "func")

        reference_filepath = ("../EW_EmpInt_behav/data/behavioraldata/")
        run_ref_filepath = os.path.join(reference_filepath, "scanfiles", sub + "_run_ref.csv")
        run_duration_filepath = os.path.join(reference_filepath, "scanfiles_derivatives/rundata/video_timing", subject + "_task-emp_run-0" + str(run) + "_events_other.txt")

        try: 
            # load reference file information
            run_ref = pd.DataFrame.from_csv(run_ref_filepath)
            run_duration = pd.DataFrame.from_csv(run_duration_filepath, sep="\t").reset_index()
            run_ref = pd.merge(run_ref, run_duration, on='video_number')

            # Get data only for relevant epi run
            run_ref = run_ref[run_ref.run == run]
            run_ref['video_number'] = run_ref['video_number'].astype(int)

            #Calculate trs
            # add three to account for hemodynamic lag
            run_ref['video_start_tr'] = \
                [round_up_to_even(x)/2 + 3 + 5 for x in run_ref['video_start_rts']] ## added + 5 to get rid of first three TRs

            run_ref['video_duration_tr'] = run_ref['duration']/2


            # Split each epi into four videos
            for index, video in enumerate(run_ref['video_number'].tolist()): 

                run_filename = "sub-" + sub + "_task-emp_run-0" + str(run) + "_bold_space-MNI152NLin2009cAsym_preproc.nii.gz"
                bold_input_filename = os.path.join(data_filepath, run_filename)

                video_filename = ("video_{video}.nii.gz").format(video=str(video))
                bold_output_filename = os.path.join(data_filepath, video_filename)

                start = run_ref.iloc[index]['video_start_tr']
                duration = run_ref.iloc[index]['video_duration_tr']

                sh_cmd = ("""fslroi \
                      {bold_input_filename}\
                      {bold_output_filename}\
                      {start}\
                      {duration}""").format(bold_input_filename=bold_input_filename, bold_output_filename=bold_output_filename, start=start, duration=duration)
                sh(sh_cmd)
                print(sh_cmd)





            # Plot Video Splitting to verify
            from nilearn import image
            from nilearn import plotting


            #Load original and cut data
            bold_input = image.smooth_img(bold_input_filename, fwhm=3)
            bold_output= image.smooth_img(bold_output_filename, fwhm=3)

            print(bold_input.shape)
            print(bold_output.shape)

            #Guess auditiory cortex seed
            aud_coords = [(-56,-4,-10),(56,-4,-10)]
            labels = ['Auditory Cortex - L', 'Auditory Cortex - R']

            #Make mask of auditory cortex
            masker = input_data.NiftiSpheresMasker(
                aud_coords, radius=8,
                detrend=True, standardize=True,
                low_pass=0.1, high_pass=0.01, t_r=2,
                memory='nilearn_cache', memory_level=1, verbose=1)


            time_series = masker.fit_transform(bold_input)

            for time_serie, label in zip(time_series.T, labels):
                plt.plot(time_serie, label=label)


            plt.axvspan(run_ref.iloc[0]['video_start_tr'],run_ref.iloc[0]['video_start_tr'] + run_ref.iloc[0]['video_duration_tr'], alpha =.2)
            plt.axvspan(run_ref.iloc[1]['video_start_tr'],run_ref.iloc[1]['video_start_tr'] + run_ref.iloc[1]['video_duration_tr'], alpha =.2)
            plt.axvspan(run_ref.iloc[2]['video_start_tr'],run_ref.iloc[2]['video_start_tr'] + run_ref.iloc[2]['video_duration_tr'], alpha =.2)
            plt.axvspan(run_ref.iloc[3]['video_start_tr'],run_ref.iloc[3]['video_start_tr'] + run_ref.iloc[3]['video_duration_tr'], alpha =.2)


            plt.title('Aud Time Series')
            plt.xlabel('TR number')
            plt.ylabel('Normalized signal')
            plt.legend()
            plt.tight_layout()


            plt.savefig(data_filepath + "/run-" +  str(run) + ".png")
            plt.gcf().clear()
            
        except:
            pass

fslroi                       ./data/sub-145/func/sub-145_task-emp_run-01_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-145/func/video_12.nii.gz                      14.0                      59.9731169945
fslroi                       ./data/sub-145/func/sub-145_task-emp_run-01_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-145/func/video_4.nii.gz                      82.0                      60.15709689
fslroi                       ./data/sub-145/func/sub-145_task-emp_run-01_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-145/func/video_8.nii.gz                      150.0                      59.7248913775
fslroi                       ./data/sub-145/func/sub-145_task-emp_run-01_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-145/func/video_16.nii.gz                      218.0                      60.090217892
(68, 80, 68, 274)
(68, 80, 68, 56)
fslroi              

fslroi                       ./data/sub-157/func/sub-157_task-emp_run-04_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-157/func/video_16.nii.gz                      82.0                      59.867688509
fslroi                       ./data/sub-157/func/sub-157_task-emp_run-04_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-157/func/video_2.nii.gz                      150.0                      57.7242028325
fslroi                       ./data/sub-157/func/sub-157_task-emp_run-04_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-157/func/video_9.nii.gz                      216.0                      43.9425593525
(68, 80, 68, 254)
(68, 80, 68, 38)
fslroi                       ./data/sub-168/func/sub-168_task-emp_run-01_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-168/func/video_10.nii.gz                      14.0                      37.4423283735
fslroi            

[NiftiSpheresMasker.transform_single_imgs] Cleaning extracted signals
_______________________________________________filter_and_extract - 3.9s, 0.1min
fslroi                       ./data/sub-184/func/sub-184_task-emp_run-01_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-184/func/video_11.nii.gz                      14.0                      60.000172356
fslroi                       ./data/sub-184/func/sub-184_task-emp_run-01_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-184/func/video_16.nii.gz                      82.0                      59.9830794475
fslroi                       ./data/sub-184/func/sub-184_task-emp_run-01_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-184/func/video_6.nii.gz                      150.0                      59.6785711425
fslroi                       ./data/sub-184/func/sub-184_task-emp_run-01_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                     

[NiftiSpheresMasker.transform_single_imgs] Cleaning extracted signals
_______________________________________________filter_and_extract - 4.0s, 0.1min
fslroi                       ./data/sub-186/func/sub-186_task-emp_run-01_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-186/func/video_16.nii.gz                      14.0                      59.906462286
fslroi                       ./data/sub-186/func/sub-186_task-emp_run-01_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-186/func/video_7.nii.gz                      82.0                      59.8833785745
fslroi                       ./data/sub-186/func/sub-186_task-emp_run-01_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-186/func/video_11.nii.gz                      150.0                      59.9629271855
fslroi                       ./data/sub-186/func/sub-186_task-emp_run-01_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                     

fslroi                       ./data/sub-200/func/sub-200_task-emp_run-02_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-200/func/video_12.nii.gz                      218.0                      59.8918595655
(68, 80, 68, 272)
(68, 80, 68, 54)
________________________________________________________________________________
[Memory] Calling nilearn.input_data.base_masker.filter_and_extract...
filter_and_extract(<nibabel.nifti1.Nifti1Image object at 0x11602b5d0>, <nilearn.input_data.nifti_spheres_masker._ExtractionFunctor object at 0x110957490>, 
{ 'allow_overlap': False,
  'detrend': True,
  'high_pass': 0.01,
  'low_pass': 0.1,
  'mask_img': None,
  'radius': 8,
  'seeds': [(-56, -4, -10), (56, -4, -10)],
  'smoothing_fwhm': None,
  'standardize': True,
  't_r': 2}, confounds=None, memory_level=1, verbose=1, memory=Memory(cachedir='nilearn_cache/joblib'))
[NiftiSpheresMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(68, 80, 68, 272),
affi

...

fslroi                       ./data/sub-222/func/sub-222_task-emp_run-04_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-222/func/video_15.nii.gz                      195.0                      60.229810898
(68, 80, 68, 249)
(68, 80, 68, 54)
________________________________________________________________________________
[Memory] Calling nilearn.input_data.base_masker.filter_and_extract...
filter_and_extract(<nibabel.nifti1.Nifti1Image object at 0x11608d0d0>, <nilearn.input_data.nifti_spheres_masker._ExtractionFunctor object at 0x105562c10>, 
{ 'allow_overlap': False,
  'detrend': True,
  'high_pass': 0.01,
  'low_pass': 0.1,
  'mask_img': None,
  'radius': 8,
  'seeds': [(-56, -4, -10), (56, -4, -10)],
  'smoothing_fwhm': None,
  'standardize': True,
  't_r': 2}, confounds=None, memory_level=1, verbose=1, memory=Memory(cachedir='nilearn_cache/joblib'))
[NiftiSpheresMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(68, 80, 68, 249),
affin

fslroi                       ./data/sub-223/func/sub-223_task-emp_run-04_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-223/func/video_4.nii.gz                      202.0                      60.1290864075
(68, 80, 68, 257)
(68, 80, 68, 55)
________________________________________________________________________________
[Memory] Calling nilearn.input_data.base_masker.filter_and_extract...
filter_and_extract(<nibabel.nifti1.Nifti1Image object at 0x1055bf5d0>, <nilearn.input_data.nifti_spheres_masker._ExtractionFunctor object at 0x1055bf850>, 
{ 'allow_overlap': False,
  'detrend': True,
  'high_pass': 0.01,
  'low_pass': 0.1,
  'mask_img': None,
  'radius': 8,
  'seeds': [(-56, -4, -10), (56, -4, -10)],
  'smoothing_fwhm': None,
  'standardize': True,
  't_r': 2}, confounds=None, memory_level=1, verbose=1, memory=Memory(cachedir='nilearn_cache/joblib'))
[NiftiSpheresMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(68, 80, 68, 257),
affin

fslroi                       ./data/sub-229/func/sub-229_task-emp_run-04_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-229/func/video_12.nii.gz                      216.0                      60.0007410875
(68, 80, 68, 271)
(68, 80, 68, 55)
________________________________________________________________________________
[Memory] Calling nilearn.input_data.base_masker.filter_and_extract...
filter_and_extract(<nibabel.nifti1.Nifti1Image object at 0x115ee6190>, <nilearn.input_data.nifti_spheres_masker._ExtractionFunctor object at 0x115ee6e50>, 
{ 'allow_overlap': False,
  'detrend': True,
  'high_pass': 0.01,
  'low_pass': 0.1,
  'mask_img': None,
  'radius': 8,
  'seeds': [(-56, -4, -10), (56, -4, -10)],
  'smoothing_fwhm': None,
  'standardize': True,
  't_r': 2}, confounds=None, memory_level=1, verbose=1, memory=Memory(cachedir='nilearn_cache/joblib'))
[NiftiSpheresMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(68, 80, 68, 271),
affi

fslroi                       ./data/sub-245/func/sub-245_task-emp_run-04_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-245/func/video_13.nii.gz                      216.0                      60.110799141
(68, 80, 68, 271)
(68, 80, 68, 55)
________________________________________________________________________________
[Memory] Calling nilearn.input_data.base_masker.filter_and_extract...
filter_and_extract(<nibabel.nifti1.Nifti1Image object at 0x115f67890>, <nilearn.input_data.nifti_spheres_masker._ExtractionFunctor object at 0x1055625d0>, 
{ 'allow_overlap': False,
  'detrend': True,
  'high_pass': 0.01,
  'low_pass': 0.1,
  'mask_img': None,
  'radius': 8,
  'seeds': [(-56, -4, -10), (56, -4, -10)],
  'smoothing_fwhm': None,
  'standardize': True,
  't_r': 2}, confounds=None, memory_level=1, verbose=1, memory=Memory(cachedir='nilearn_cache/joblib'))
[NiftiSpheresMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(68, 80, 68, 271),
affin

fslroi                       ./data/sub-253/func/sub-253_task-emp_run-04_bold_space-MNI152NLin2009cAsym_preproc.nii.gz                      ./data/sub-253/func/video_3.nii.gz                      218.0                      59.9098775595
(68, 80, 68, 272)
(68, 80, 68, 54)
________________________________________________________________________________
[Memory] Calling nilearn.input_data.base_masker.filter_and_extract...
filter_and_extract(<nibabel.nifti1.Nifti1Image object at 0x115f67310>, <nilearn.input_data.nifti_spheres_masker._ExtractionFunctor object at 0x115fcb850>, 
{ 'allow_overlap': False,
  'detrend': True,
  'high_pass': 0.01,
  'low_pass': 0.1,
  'mask_img': None,
  'radius': 8,
  'seeds': [(-56, -4, -10), (56, -4, -10)],
  'smoothing_fwhm': None,
  'standardize': True,
  't_r': 2}, confounds=None, memory_level=1, verbose=1, memory=Memory(cachedir='nilearn_cache/joblib'))
[NiftiSpheresMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(68, 80, 68, 272),
affin